In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim

In [2]:
torch.manual_seed(42)

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
data=pd.read_excel('/content/Bok1.xlsx')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,Subscription
0,56.0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57.0,services,married,high.school,unknown,no,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37.0,services,married,high.school,no,yes,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40.0,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56.0,services,married,high.school,no,no,yes,telephone,may,mon,...,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
data.shape

(95041, 21)

In [6]:
data['Subscription'].value_counts()

,count
Subscription,
no,40215
yes,5091
y,1


In [7]:
data[data['Subscription'] == 'y']


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,Subscription
86400,NaN,job,marital,education,default,housing,loan,contact,month,day_of_week,...,NaN,NaN,NaN,poutcome,NaN,NaN,NaN,NaN,NaN,y


In [8]:
data.drop(index=86400,axis=0,inplace=True)

In [9]:
data[data['Subscription'] == 'y']


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,Subscription


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95040 entries, 0 to 95040
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             95038 non-null  float64
 1   job             95040 non-null  object 
 2   marital         95040 non-null  object 
 3   education       95040 non-null  object 
 4   default         95040 non-null  object 
 5   housing         95040 non-null  object 
 6   loan            95040 non-null  object 
 7   contact         95040 non-null  object 
 8   month           95040 non-null  object 
 9   day_of_week     95040 non-null  object 
 10  duration        45306 non-null  float64
 11  campaign        95038 non-null  float64
 12  pdays           95038 non-null  float64
 13  previous        95038 non-null  float64
 14  poutcome        95040 non-null  object 
 15  emp.var.rate    45306 non-null  float64
 16  cons.price.idx  45306 non-null  float64
 17  cons.conf.idx   45306 non-null  floa

In [11]:
data.isnull().sum()

,0
age,2
job,0
marital,0
education,0
default,0
housing,0
loan,0
contact,0
month,0
day_of_week,0


In [12]:
data.dropna(inplace=True)

In [13]:
# Numerical and categorical columns
num_cols = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx',
            'cons.conf.idx', 'euribor3m', 'nr.employed']
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan',
            'contact', 'month', 'day_of_week', 'poutcome']

# Filling numerical columns with median
for col in num_cols:
    data[col] = data[col].fillna(data[col].median())

# Fill categorical columns with 'Unknown'
for col in cat_cols:
    data[col] = data[col].fillna('Unknown')

# Handle missing target values
data['Subscription'] = data['Subscription'].fillna(data['Subscription'].mode()[0])

In [14]:
data.isnull().sum()

,0
age,0
job,0
marital,0
education,0
default,0
housing,0
loan,0
contact,0
month,0
day_of_week,0


In [15]:
data['Subscription'].unique()

array(['no', 'yes'], dtype=object)

In [16]:
data['Subscription'] = data['Subscription'].replace({'no': 0, 'yes': 1})
data = data.infer_objects(copy=False)


<ipython-input-16-3a1c21872f36>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Subscription'] = data['Subscription'].replace({'no': 0, 'yes': 1})


In [17]:
data['Subscription'].unique()

array([0, 1])

In [18]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [19]:
X.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56.0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261.0,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,57.0,services,married,high.school,unknown,no,no,telephone,may,mon,149.0,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,37.0,services,married,high.school,no,yes,no,telephone,may,mon,226.0,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,40.0,admin.,married,basic.6y,no,no,no,telephone,may,mon,151.0,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,56.0,services,married,high.school,no,no,yes,telephone,may,mon,307.0,1.0,999.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


In [20]:
y.head()

,Subscription
0,0
1,0
2,0
3,0
4,0


In [21]:
 import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#  Dropping unnecessary columns
columns_to_drop = ['job', 'education', 'contact', 'month', 'duration', 'pdays', 'previous',
                   'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

X = X.drop(columns=columns_to_drop, errors='ignore')


#  Label Encoding for Binary Columns
binary_cols = ['default', 'housing', 'loan']
le_binary = LabelEncoder()
for col in binary_cols:
    X[col] = le_binary.fit_transform(X[col])

#  Label Encoding for Ordinal Categorical Columns
label_enc_cols = ['marital', 'poutcome']
label_encoders = {}
for col in label_enc_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

#   One-Hot Encoding for Nominal Categorical Variables
one_hot_cols = ['day_of_week']
X = pd.get_dummies(X, columns=one_hot_cols, drop_first=True)
print("One-Hot Encoding Applied:\n", X.head())
X = X.astype(int)


#  Standardizing All Numerical Columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])



One-Hot Encoding Applied:
     age  marital  default  housing  loan  campaign  poutcome  day_of_week_mon  \
0  56.0        1        0        0     0       1.0         1             True   
1  57.0        1        1        0     0       1.0         1             True   
2  37.0        1        0        2     0       1.0         1             True   
3  40.0        1        0        0     0       1.0         1             True   
4  56.0        1        0        0     2       1.0         1             True   

   day_of_week_thu  day_of_week_tue  day_of_week_wed  
0            False            False            False  
1            False            False            False  
2            False            False            False  
3            False            False            False  
4            False            False            False  


In [22]:
X.head()

,age,marital,default,housing,loan,campaign,poutcome,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,1.533677,-0.284485,-0.511659,-1.088741,-0.454079,-0.568568,0.193602,1.958503,-0.514508,-0.495506,-0.495436
1,1.629727,-0.284485,1.952331,-1.088741,-0.454079,-0.568568,0.193602,1.958503,-0.514508,-0.495506,-0.495436
2,-0.291263,-0.284485,-0.511659,0.941365,-0.454079,-0.568568,0.193602,1.958503,-0.514508,-0.495506,-0.495436
3,-0.003114,-0.284485,-0.511659,-1.088741,-0.454079,-0.568568,0.193602,1.958503,-0.514508,-0.495506,-0.495436
4,1.533677,-0.284485,-0.511659,-1.088741,2.303443,-0.568568,0.193602,1.958503,-0.514508,-0.495506,-0.495436


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
#creating a custom class
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    # Convert features to NumPy array before creating a tensor
    self.features=torch.tensor(features.values,dtype=torch.float32)
    self.labels=torch.tensor(labels.values,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [25]:
#creating dataset object class for traaining and testing
train_dataset=CustomDataset(X_train,y_train)
test_dataset=CustomDataset(X_test,y_test)

In [26]:
#creating train and test loader
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=128,shuffle=False,pin_memory=True)


In [27]:
len(train_loader)

284

In [28]:
len(test_loader)

71

In [29]:
#custom nn model
#custom nn model
class MyNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    )
  def forward(self,x):
    return self.model(x)

In [30]:
learning_rate=0.01
epochs=10

In [31]:
#instatiate the model
model=MyNN(X_train.shape[1])
model=model.to(device)
#loss function
criterion=nn.CrossEntropyLoss()
#optimizer
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [32]:
#training loop
for epoch in range(epochs):
  total_epoch_loss=0

  for batch_features,batch_labels in train_loader:
    #move data to gpu
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    #forward pass
    output=model(batch_features)
    #calculate loss
    loss=criterion(output,batch_labels)
    #back pass
    optimizer.zero_grad()
    loss.backward

    #update weights
    optimizer.step()
    total_epoch_loss+=loss.item()
  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_epoch_loss/len(train_loader)}")

Epoch 1/10, Loss: 0.6682811173754679
Epoch 2/10, Loss: 0.6681565413592567
Epoch 3/10, Loss: 0.6682388616699568
Epoch 4/10, Loss: 0.6681944513824624
Epoch 5/10, Loss: 0.6682072397268993
Epoch 6/10, Loss: 0.6682114185581745
Epoch 7/10, Loss: 0.6681693299135691
Epoch 8/10, Loss: 0.6682471425180704
Epoch 9/10, Loss: 0.6681633113135754
Epoch 10/10, Loss: 0.668207649194019


In [33]:
#setting the model to evaluate mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [34]:
#evaluation
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    #move data to gpu
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    outputs=model(batch_features)
    _,predicted=torch.max(outputs.data,1)
    total+=batch_labels.size(0)
    correct+=(predicted==batch_labels).sum().item()
  print(f"Accuracy: {100*correct/total}")

Accuracy: 71.15427058044581


In [35]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(100*correct/total)

71.1593643085752
